In [5]:
import sys
import psycopg2
import pandas as pd
from sqlalchemy import create_engine
from psycopg2 import OperationalError, errorcodes, errors
from configdef import *

In [3]:
def show_psycopg2_exception(err):
    # get details about the exception
    err_type, err_obj, traceback = sys.exc_info()    # get the line number when exception occured
    line_n = traceback.tb_lineno    # print the connect() error
    print ("\npsycopg2 ERROR:", err, "on line number:", line_n)
    print ("psycopg2 traceback:", traceback, "-- type:", err_type)    # psycopg2 extensions.Diagnostics object attribute
    print ("\nextensions.Diagnostics:", err.diag)    # print the pgcode and pgerror exceptions
    print ("pgerror:", err.pgerror)
    print ("pgcode:", err.pgcode, "\n")

In [9]:
def connect(conn_params_dic):
    conn = None
    try:
        print('Connecting to the PostgreSQL...........')
        conn = psycopg2.connect(**conn_params_dic)
        print("Connection successful..................")
        
    except OperationalError as err:
        # passing exception to function
        show_psycopg2_exception(err)        # set the connection to 'None' in case of error
        conn = None
    
    return conn

In [35]:
params = config(section='postgres')
conn = connect(params)
conn.autocommit = True
if conn!=None:
    
    try:
        cursor = conn.cursor();
        # Dropping database airline if exists
        cursor.execute("DROP SCHEMA IF EXISTS airline;")
    
        # Creating a database
        cursor.execute("CREATE SCHEMA airline;");
        print("airline database is created successfully............")
    
        # Closing the cursor & connection
        cursor.close()
        conn.close()
        
    except OperationalError as err:
        # pass exception to function
        show_psycopg2_exception(err)
        # set the connection to 'None' in case of error
        conn = None

Connecting to the PostgreSQL...........
Connection successful..................
airline database is created successfully............


In [38]:
create_airline_db_sql = ('''
    
    DROP TABLE IF EXISTS flights CASCADE;

    CREATE TABLE flights (  
      year smallint DEFAULT NULL,
      month smallint DEFAULT NULL,
      day smallint DEFAULT NULL,
      flightdate date DEFAULT NULL,
      dep_time varchar(4) NOT NULL DEFAULT '',
      sched_dep_time smallint DEFAULT NULL,
      dep_delay smallint DEFAULT NULL,
      arr_time varchar(4) NOT NULL DEFAULT '',
      sched_arr_time smallint DEFAULT NULL,
      arr_delay smallint DEFAULT NULL,
      carrier varchar(2) NOT NULL DEFAULT '',
      tailnum varchar(6) DEFAULT NULL,
      flight smallint DEFAULT NULL,
      origin varchar(3) NOT NULL DEFAULT '',
      dest varchar(3) NOT NULL DEFAULT '',
      air_time smallint DEFAULT NULL,
      distance smallint DEFAULT NULL,
      cancelled smallint DEFAULT NULL,
      diverted smallint DEFAULT NULL
    );

    CREATE INDEX IF NOT EXISTS year_idx ON flights (year);
    CREATE INDEX IF NOT EXISTS date_idx ON flights (year, month, day);
    CREATE INDEX IF NOT EXISTS origin_idx ON flights (origin);
    CREATE INDEX IF NOT EXISTS dest_idx ON flights (dest);
    CREATE INDEX IF NOT EXISTS carrier_idx ON flights (carrier);
    CREATE INDEX IF NOT EXISTS tailnum_idx ON flights (tailnum);

    DROP TABLE IF EXISTS carriers;

    CREATE TABLE carriers (
      carrier varchar(7) NOT NULL DEFAULT '',
      name varchar(255) NOT NULL DEFAULT '',
      PRIMARY KEY (carrier)
    );

    DROP TABLE IF EXISTS airports;

    CREATE TABLE airports (
      faa varchar(3) NOT NULL DEFAULT '',
      name varchar(255),
      lat decimal(10,7) DEFAULT NULL,
      lon decimal(10,7) DEFAULT NULL,
      alt int DEFAULT NULL,
      tz smallint DEFAULT NULL,
      dst char(1),
      city varchar(255) DEFAULT NULL,
      country varchar(255) DEFAULT NULL,
      PRIMARY KEY (faa)
    );

    DROP TABLE IF EXISTS planes;

    CREATE TABLE planes (
      tailnum varchar(6) NOT NULL DEFAULT '',
      year int DEFAULT NULL,
      type text,
      manufacturer text,
      model text,
      engines int DEFAULT NULL,
      seats int DEFAULT NULL,
      speed int DEFAULT NULL,
      engine text,
      PRIMARY KEY (tailnum)
    );
                         ''')

In [47]:
create_flights_table_sql = ('''
    
    DROP TABLE IF EXISTS flights CASCADE;

    CREATE TABLE flights (  
      year smallint DEFAULT NULL,
      month smallint DEFAULT NULL,
      day smallint DEFAULT NULL,
      flightdate date DEFAULT NULL,
      dep_time smallint DEFAULT NULL,
      sched_dep_time smallint DEFAULT NULL,
      dep_delay smallint DEFAULT NULL,
      arr_time smallint DEFAULT NULL,
      sched_arr_time smallint DEFAULT NULL,
      arr_delay smallint DEFAULT NULL,
      carrier varchar(2) NOT NULL DEFAULT '',
      tailnum varchar(6) DEFAULT NULL,
      flight smallint DEFAULT NULL,
      origin varchar(3) NOT NULL DEFAULT '',
      dest varchar(3) NOT NULL DEFAULT '',
      air_time smallint DEFAULT NULL,
      distance smallint DEFAULT NULL,
      cancelled smallint DEFAULT NULL,
      diverted smallint DEFAULT NULL
    );

    CREATE INDEX IF NOT EXISTS year_idx ON flights (year);
    CREATE INDEX IF NOT EXISTS date_idx ON flights (year, month, day);
    CREATE INDEX IF NOT EXISTS origin_idx ON flights (origin);
    CREATE INDEX IF NOT EXISTS dest_idx ON flights (dest);
    CREATE INDEX IF NOT EXISTS carrier_idx ON flights (carrier);
    CREATE INDEX IF NOT EXISTS tailnum_idx ON flights (tailnum);
''')

In [49]:
params = config(section='postgres')
conn = connect(params)
conn.autocommit = True
if conn!=None:
    
    try:
        cursor = conn.cursor();
        # Running airline table creatiuon sql
        cursor.execute(create_flights_table_sql)
        conn.close()
        
    except OperationalError as err:
        # pass exception to function
        show_psycopg2_exception(err)
        # set the connection to 'None' in case of error
        conn = None

Connecting to the PostgreSQL...........
Connection successful..................
